## 1

In [1]:
#url <- "https://raw.githubusercontent.com/qwerty29544/RpracticeBook/master/2Data/01FlatTables/ECG_yurchenkov.txt"
#download_folder <- "./data_input/"
#
#download.file(url, paste(download_folder, "ECG_yurchenkov.txt", sep=""))

## 2

In [2]:
extract_variables <- function(block, vars) {
    stopifnot(is.vector(block) & is.character(block))
    stopifnot(is.vector(vars) & is.character(vars))
    
    result <- c()
    
    index <- 0
    variable <- ''
    for (name in vars) {
        index <- grep(paste('<', name, '>', sep = ''), block)
        variable <- sub(paste(' <', name, '>', sep = ''), '', block[index])
        result <- c(result, variable)
    }
    
    return (result)
}

In [3]:
parse_track_info <- function(strings) {
    stopifnot(is.vector(strings) & is.character(strings))
    
    variables <- extract_variables(strings, c('Аббревиатура канала', 'Номер трека', 'Тип трека', 
                                              'Единица измерения', 'Частота дискретизации, Гц'))
    track_info <- list(
        abbr = variables[1],
        number = as.integer(variables[2]),
        type = variables[3],
        unit = variables[4],
        frequency = as.numeric(variables[5])
    )
    class(track_info) <- 'TrackInfo'
    
    return (track_info)    
}

In [4]:
parse_fragment <- function(strings) {
    stopifnot(is.vector(strings) & is.character(strings))
    
    variables <- extract_variables(strings[1:5], c('Название фрагмента', 'Время начала фрагмента от начала исследования, в мс',
                                              'Длина фрагмента, в мс', 'Длина фрагмента, в отсчетах'))
    
    data <- read.table(text = strings[6:length(strings)], sep = "\t")
    colnames(data) <- data[1,]
    data <- data[-1, ]
    rownames(data) <- NULL

    fragment <- list(
        name = variables[1],
        start_time = as.numeric(variables[2]),
        time_length = as.numeric(variables[3]),
        counts_length = as.numeric(variables[4]),
        
        data = data
    )
    class(fragment) <- 'Fragment'
    
    return (fragment)
}

In [5]:
parse_ecg <- function(file_name, rename_columns = TRUE, unique_colnames = TRUE) {
    stopifnot(is.character(file_name))
    stopifnot(is.logical(rename_columns))
    stopifnot(is.logical(unique_colnames))
    
    all_lines <- readLines(file_name)
    
    variables <- extract_variables(all_lines[1:5], c('Группа', 'Наименование', 'Число треков', 'Число фрагментов'))
    
    tracks_end <- 6
    for (i in 6:length(all_lines)) {
        if (startsWith(all_lines[i], '// Информация по фрагменту')) {
            tracks_end <- i
            break
        }
    }
    tracks_bounds <- c()
    for (i in 6:tracks_end) {
        if (startsWith(all_lines[i], '// Информация о треке')) {
            tracks_bounds <- c(tracks_bounds, i)
        }
    }
    tracks_bounds <- c(tracks_bounds, tracks_end)
    
    tracks_infos <- list()
    for (i in 1:(length(tracks_bounds) - 1)) {
        from <- tracks_bounds[i]
        to <- tracks_bounds[i + 1] - 1
        tracks_infos[[i]] <- parse_track_info(all_lines[from:to])
    }
    
    fragment_bounds <- c()
    for (i in 1:length(all_lines)) {
        if (startsWith(all_lines[i], '// Информация по фрагменту')) {
            fragment_bounds <- c(fragment_bounds, i)
        }
    }
    fragment_bounds <- c(fragment_bounds, length(all_lines))
    
    fragments <- list()
    for (i in 1:(length(fragment_bounds) - 1)) {
        from <- fragment_bounds[i]
        to <- fragment_bounds[i + 1] - 1
        fragments[[i]] <- parse_fragment(all_lines[from:to])
    }
    
    if (rename_columns) {
        col_names <- sapply(tracks_infos, function (x) x$abbr)
        col_names <- c('Время(мс)', col_names)
        if (unique_colnames) {
            col_names <- make.unique(col_names)
        }
        
        for (i in 1:length(fragments)) {
            colnames(fragments[[i]]$data) <- col_names
        }
    }
    
    ecg <- list(
        group = variables[1],
        name = variables[2],
        tracks_count = as.integer(variables[3]),
        fragments_count = as.integer(variables[4])
    )
    ecg$tracks <- tracks_infos
    ecg$fragments <- fragments
    
    class(ecg) <- 'EKG'
    
    return (ecg)
}

Как же я заебался парсить эту хрень...

In [6]:
ecg <- parse_ecg('./data_input/ECG_yurchenkov.txt', rename_columns = TRUE)

In [7]:
head(ecg$fragments[[1]])

Время(мс),ЭКГ - I,ФПГ,ФПГ.1,КГР R_R,Абдом.
0,"-0,02875","-0,024375","0,3125","-1,44196","1,475"
4,"-0,018","-0,023125","0,3125","-1,44196","1,5125"
8,"0,02975","-0,021875","0,2375","-1,44196","1,5375"
12,"-0,01025","-0,02125","0,0875","-1,44043","1,575"
16,"-0,1215","-0,02125","-0,1","-1,4389","1,6125"
20,"-0,1375","-0,021875","-0,3375","-1,4389","1,65"
24,"-0,07525","-0,02375","-0,5625","-1,4389","1,6875"
28,"-0,07475","-0,02625","-0,75","-1,4389","1,7125"
32,"-0,105","-0,029375","-0,9","-1,4389","1,75"
36,"-0,086","-0,033125","-0,975","-1,43738","1,7875"


In [8]:
data <- ecg$fragments[[1]]$data

In [9]:
head(data)

Время(мс),ЭКГ - I,ФПГ,ФПГ.1,КГР R_R,Абдом.
0,"-0,02875","-0,024375","0,3125","-1,44196","1,475"
4,"-0,018","-0,023125","0,3125","-1,44196","1,5125"
8,"0,02975","-0,021875","0,2375","-1,44196","1,5375"
12,"-0,01025","-0,02125","0,0875","-1,44043","1,575"
16,"-0,1215","-0,02125","-0,1","-1,4389","1,6125"
20,"-0,1375","-0,021875","-0,3375","-1,4389","1,65"


In [10]:
x <- data[,1][seq(1, length(data[,1]), 50)]
y <- data[,2][seq(1, length(data[,2]), 50)]